# Rancang Bangun Pengendali Otomatis Lingkungan Termal Berbasis Jaringan Saraf Tiruan

### Program Kecerdasan Buatan untuk membentuk model jaringan saraf tiruan dalam memenuhi Tugas Akhir program studi Teknik Fisika.
### Data di Bulan Maret 2019

In [1]:
# Mengimpor pustaka yang dibutuhkan.

import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
# Memasukkan dataset ke dalam jupyter.
data = pd.read_excel('../Data/dataAll.xlsx')

In [3]:
# Menampilkan 5 data teratas.
data.head()

,No,Variation,Month,Time,Heater,AC,DrybulbT,Radiation,AirT,RH
0,1,DT001,Mar,00:03:00,0,0,24.08,0.0,24.03,70.64
1,2,DT001,Mar,00:09:00,0,0,24.08,0.0,24.03,70.72
2,3,DT001,Mar,00:15:00,0,0,24.08,0.0,24.04,70.81
3,4,DT001,Mar,00:21:00,0,0,24.08,0.0,24.04,70.90
4,5,DT001,Mar,00:27:00,0,0,24.08,0.0,24.04,70.99


In [4]:
# Membuang fitur No, dan Variation.
data = data.drop(['No','Variation'], axis =1)

In [5]:
# # Mengubah tipe data waktu
# def ubahT(dt):
#     waktu = 60*int(dt.strftime('%H')) + int(dt.strftime('%M'))
#     return waktu

# data['Time'] = data['Time'].apply(ubahT)
# data['Time']

In [6]:
# # Mengubah data Month

# def ubahM(dt):
#     if dt == 'Mar':
#         return 3
#     elif dt == 'Jun':
#         return 6
#     elif dt == 'Sep':
#         return 9
#     elif dt == 'Des':
#         return 12

# data['Month'] = data['Month'].apply(ubahM)
# data['Month']

In [7]:
# Memisahkan data input dan data target.

# Data Input
X = data[['Heater','AC','DrybulbT','Radiation']]

# Data Target
target = data[['AirT','RH']]

In [8]:
# Memisahkan data pelatihan (training) dan data pengujian (testing).

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Memisahkan 20% menjadi data tes
X_temp, X_test, y_temp, y_test = train_test_split(X, target, test_size=0.2, shuffle=True, random_state=15)

# Memisahkan 20% menjadi data validation
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, shuffle=False, random_state=2019)

In [9]:
# Menghapus data temporary
del X_temp, y_temp

# print proportions
print('train: {}% | validation: {}% | test {}%'.format(round(len(y_train)/len(target)*100,2),
                                                       round(len(y_val)/len(target)*100,2),
                                                       round(len(y_test)/len(target)*100,2)))

train: 0.6% | validation: 0.2% | test 0.2%


In [10]:
# banyak data
print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)

(14400, 4) (4800, 4) (4800, 4) (14400, 2) (4800, 2) (4800, 2)


## Model Tuning menggunakan Multi-Layer Perceptron Regressor

In [11]:
# Impor pustaka
from sklearn.neural_network import MLPRegressor

In [30]:
# Mencari optimasi jumlah hidden layer dan neuron terbaik
import operator

def ann(m):
    return MLPRegressor(random_state = 1, activation = 'relu', hidden_layer_sizes=(m), solver='adam', alpha=0.001, max_iter = 5000)

daftar = {}
for i in range(1,21):
    model = ann(i)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred)**0.5
    r2 = r2_score(y_val, y_pred)
    key = str(i)
    value = str(rmse) + ',' + str(r2)
    daftar[key] = rmse
    print(i,': ','RMSE =',rmse,', R^2 =',r2)
    if rmse < 6.101:
        neuron = {i}
        break
    else:
        neuron = min(daftar.items(), key=operator.itemgetter(1))[0]

print(neuron, daftar)  # 20, 4 : RMSE = 6,19...

1 :  RMSE = 7.676472390893648 , R^2 = -0.00026894498026275304
2 :  RMSE = 7.601981619207993 , R^2 = -0.02186397924854777
3 :  RMSE = 6.914552776288646 , R^2 = 0.10266266746220953
4 :  RMSE = 7.625446737559507 , R^2 = -0.02242161963257261
5 :  RMSE = 6.908410476341447 , R^2 = 0.23582245803481527
6 :  RMSE = 6.819693082556216 , R^2 = 0.262550516548783
7 :  RMSE = 6.882281935858971 , R^2 = 0.26835336354660305
8 :  RMSE = 6.660913806747602 , R^2 = 0.41102941289499284
9 :  RMSE = 6.582485375620983 , R^2 = 0.46360169231119835
10 :  RMSE = 6.377444146443421 , R^2 = 0.5375922595511765
11 :  RMSE = 6.4337210646539695 , R^2 = 0.4954351579137487
12 :  RMSE = 6.7997344837035705 , R^2 = 0.29657237834788247
13 :  RMSE = 6.4902747969859576 , R^2 = 0.5005818823519589
14 :  RMSE = 6.361577001921606 , R^2 = 0.5449765781903044
15 :  RMSE = 6.4658094184682 , R^2 = 0.502451186197141
16 :  RMSE = 6.387860430634295 , R^2 = 0.5447046862836948
17 :  RMSE = 6.756557820451211 , R^2 = 0.3037549340057641
18 :  RMS

In [29]:
# Mencari optimasi jumlah hidden layer dan neuron terbaik
import operator

def ann(m,n):
    return MLPRegressor(random_state = 1, activation = 'relu', hidden_layer_sizes=(m,n), solver='adam', alpha=0.001, max_iter = 5000)

daftar = {}
for i in range(20,21):
    for j in range(13,21):
        model = ann(i,j)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred)**0.5
        r2 = r2_score(y_val, y_pred)
        key = str(i) + ',' + str(j)
        value = str(rmse) + ',' + str(r2)
        daftar[key] = rmse
        print(i,j,': ','RMSE =',rmse,', R^2 =',r2)
        if rmse < 6.07:
            neuron = {i,j}
            break
        else:
            neuron = min(daftar.items(), key=operator.itemgetter(1))[0]

print(neuron, daftar)  # 20, 13 : RMSE = 6,07...

20 13 :  RMSE = 6.070160239167145 , R^2 = 0.5627795722053989
20 14 :  RMSE = 6.532899323968007 , R^2 = 0.5300336797252206
20 15 :  RMSE = 6.351689796305892 , R^2 = 0.5474061392349541
20 16 :  RMSE = 6.277376873845304 , R^2 = 0.549599404592846
20 17 :  RMSE = 6.2846640224956385 , R^2 = 0.5442819839177151
20 18 :  RMSE = 6.239618853454412 , R^2 = 0.5670320127706252
20 19 :  RMSE = 6.446004638272175 , R^2 = 0.524307701085486
20 20 :  RMSE = 6.247250477995721 , R^2 = 0.5427788161324067
20,13 {'20,13': 6.070160239167145, '20,14': 6.532899323968007, '20,15': 6.351689796305892, '20,16': 6.277376873845304, '20,17': 6.2846640224956385, '20,18': 6.239618853454412, '20,19': 6.446004638272175, '20,20': 6.247250477995721}


In [25]:
alphas = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10]
print('Semua error dalam nilai RMSE')
print('-'*76)
for alpha in alphas:
    # Pembuatan model
    # model_mlp = MLPRegressor(random_state=1, activation='relu', hidden_layer_sizes=(24,24,24), solver='adam')
    model_mlp = MLPRegressor(random_state = 1, activation = 'relu', hidden_layer_sizes=(20, 13), solver='adam', 
                             alpha=alpha, max_iter = 5000)

    # Melakukan pelatihan (training) menggunakan 'fit'
    model_mlp.fit(X_train, y_train)

    # Menghitung nilai RMSE
    rmse_train = mean_squared_error(y_train, model_mlp.predict(X_train))**0.5
    rmse_validation = mean_squared_error(y_val, model_mlp.predict(X_val))**0.5
    rmse_test = mean_squared_error(y_test, model_mlp.predict(X_test))**0.5

    # print errors as report
    print('alpha: {:7} | train error: {:5} | val error: {:6} | test error: {}'.
          format(alpha,
                 round(rmse_train,3),
                 round(rmse_validation,3),
                 round(rmse_test,3)))

Semua error dalam nilai RMSE
----------------------------------------------------------------------------
alpha:   1e-05 | train error: 6.123 | val error:  6.102 | test error: 6.151
alpha:  0.0001 | train error: 6.327 | val error:  6.335 | test error: 6.355
alpha:   0.001 | train error: 6.104 | val error:   6.07 | test error: 6.137
alpha:    0.01 | train error:  6.28 | val error:  6.236 | test error: 6.284
alpha:     0.1 | train error: 6.491 | val error:  6.461 | test error: 6.53
alpha:       1 | train error: 6.475 | val error:  6.412 | test error: 6.462
alpha:      10 | train error: 6.731 | val error:  6.738 | test error: 6.752


In [26]:
# Model final
model_mlp = MLPRegressor(random_state = 1, activation = 'relu', hidden_layer_sizes=(20, 13), solver='adam', 
                             alpha=0.001, max_iter = 5000)

# Melakukan pelatihan (training) menggunakan 'fit'
model_mlp.fit(X_train, y_train)

# Testing
rmse_test = mean_squared_error(y_test, model_mlp.predict(X_test))**0.5
print('test error: {}'.format(round(rmse_test,3)))

test error: 6.137


In [20]:
print(model_mlp.get_params)

<bound method BaseEstimator.get_params of MLPRegressor(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(24, 24, 24), learning_rate='constant',
             learning_rate_init=0.001, max_iter=5000, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=1, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)>


## Visualisasi

In [ ]:
# Variabel untuk pembuatan grafik

Heater    = X_test['Heater']
AC        = X_test['AC']
DrybulbT  = X_test['DrybulbT']
Radiation = X_test['Radiation']

AirT      = Y_test['AirT']
RH        = Y_test['RH']

In [ ]:
# plot a line, a perfit predict would all fall on this line
plt.subplot(2, 1, 1)
plt.title('Grafik Heater terhadap AirT dan RH')
plt.plot(X_test.values[:,0], Y_test.values[:,0], '.')
plt.plot(X_test.values[:,0], Y_pred[:,0], 'r.')
plt.ylabel('AirT')

plt.subplot(2, 1, 2)
plt.plot(X_test.values[:,0], Y_test.values[:,1], '.')
plt.plot(X_test.values[:,0], Y_pred[:,1], 'r.')
plt.xlabel('Heater')
plt.ylabel('RH')

plt.show()

In [ ]:
plt.subplot(2, 1, 1)
plt.title('Grafik AC terhadap AirT dan RH')
plt.plot(X_test.values[:,1], Y_test.values[:,0], '.')
plt.plot(X_test.values[:,1], Y_pred[:,0], 'r.')
plt.ylabel('AirT')

plt.subplot(2, 1, 2)
plt.plot(X_test.values[:,1], Y_test.values[:,1], '.')
plt.plot(X_test.values[:,1], Y_pred[:,1], 'r.')
plt.xlabel('AC')
plt.ylabel('RH')

plt.show()

In [ ]:
plt.subplot(2, 1, 1)
plt.title('Grafik DrybulbT terhadap AirT dan RH')
plt.plot(X_test.values[:,2], Y_test.values[:,0], '.')
plt.plot(X_test.values[:,2], Y_pred[:,0], 'r.')
plt.ylabel('AirT')

plt.subplot(2, 1, 2)
plt.plot(X_test.values[:,2], Y_test.values[:,1], '.')
plt.plot(X_test.values[:,2], Y_pred[:,1], 'r.')
plt.xlabel('DrybulbT')
plt.ylabel('RH')

plt.show()

In [ ]:
plt.subplot(2, 1, 1)
plt.title('Grafik Radiation terhadap AirT dan RH')
plt.plot(X_test.values[:,3], Y_test.values[:,0], '.')
plt.plot(X_test.values[:,3], Y_pred[:,0], 'r.')
plt.ylabel('AirT')

plt.subplot(2, 1, 2)
plt.plot(X_test.values[:,3], Y_test.values[:,1], '.')
plt.plot(X_test.values[:,3], Y_pred[:,1], 'r.')
plt.xlabel('Radiation')
plt.ylabel('RH')

plt.show()